## Introducción al Machine Learning 
### Cross-validation 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
bank = pd.read_csv('../data/bank.csv', sep=";")

In [ ]:
raw_features = bank.drop(columns='y')
features = pd.get_dummies(raw_features).values
target = bank.y
features

In [ ]:
target.value_counts()

Proporción de clase.
Fijaros el accuracy que obtendría un clasificador por defectom

In [ ]:
target.value_counts()/target.shape[0]

___

Entrenameros arboles de decisión

In [ ]:
tree = DecisionTreeClassifier()

Validación cruzada con la función cross_validate

In [ ]:
# Ver otras metricas disponibles con
# sklearn.metrics.SCORERS.keys()
scoring = ['accuracy', 'roc_auc']

In [ ]:
scores = cross_validate(tree, features, target, scoring=scoring, cv=4)
pd.DataFrame(scores)

In [ ]:
pd.DataFrame(scores).mean()

Otra alternativa más flexible es iterar con los indices en las particiones de la función *KFold*

In [ ]:
kfold = KFold(n_splits=4, shuffle=True)
acc = []
auc = []
for train, test in kfold.split(features):
    
    fold_train_x = features[train]
    fold_train_y = target[train]
    tree.fit(fold_train_x, fold_train_y)
    
    fold_test_x = features[test]
    fold_test_y = target[test]
    
    i_acc = tree.score(fold_test_x, fold_test_y)
    
    i_class = np.where(tree.classes_ == 'yes')[0][0]
    print('class:', i_class)
    fold_test_pred = tree.predict_proba(fold_test_x)[:,i_class]
    i_auc = roc_auc_score(fold_test_y, fold_test_pred)

    auc.append(i_auc)
    acc.append(i_acc)

In [ ]:
result = pd.DataFrame({'auc':auc, 
                       'accuracy':acc})
result

In [ ]:
result.mean()

___

#### Propuesta Ejercicio
1. Probar la validación cruzada calculando una métrica alternativa, y probando con otro número de partes.
2. Razonar las adaptaciones que tendríamos que hacer para entrenar con un KNN, en el que tenemos que estandarizar las varibales
